In [ ]:
!uv add python-dotenv instructor pydantic jsonref pandas

⠇ newspaper3k==0.2.8                                                            ⠋ Resolving dependencies...                                                     

In [9]:
import os
from dotenv import load_dotenv
import instructor
from pydantic import BaseModel, Field
from typing import Optional, List

# Load environment variables from .env file
load_dotenv()

# Get the Gemini API key from environment
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

if GEMINI_API_KEY:
    # Initialize Instructor client with Gemini
    # Instructor uses the API key from environment or can be passed directly
    os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY
    client = instructor.from_provider(
        "google/gemini-2.5-flash",
        api_key=GEMINI_API_KEY
    )
    print("Instructor client initialized successfully with Gemini!")
else:
    print("Warning: GEMINI_API_KEY not found in .env file")
    client = None

GEMINI_MODELS = [
    'gemini-2.5-flash',
    'gemini-3-pro-preview',
    'gemini-3-flash-preview',
    'gemini-flash-lite-latest',
]

Instructor client initialized successfully with Gemini!


In [2]:
import pandas as pd

In [3]:
import sqlite3

# Connect to the SQLite database
db_path = "../instance/violence.db"
conn = sqlite3.connect(db_path)

# Get the table names
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
table_names = tables['name'].tolist()

dfs = {}  # Dictionary to hold DataFrames for each table

for table in table_names:
    dfs[table] = pd.read_sql_query(f"SELECT * FROM {table};", conn)

conn.close()

# Now, dfs is a dictionary where keys are table names and values are DataFrames


In [4]:
dfs.keys()

dict_keys(['source', 'incident', 'extracted_event', 'keyword', 'alembic_version', 'lost_and_found', 'events_ground_truth'])

In [5]:
dfs['events_ground_truth']

,id,source_id,incident_id,confidence_score,extracted_date,extracted_location,extracted_victim_name,summary,death_count,group_id
0,1,646,70,1.00,2025-10-25 00:00:00,"comunidades da Penha e do Alemão, Rio de Janeiro",None,Uma megaoperação policial nas comunidades da P...,NaN,70
1,2,1372,70,1.00,2025-10-28 00:00:00,"Complexos do Alemão e da Penha, Zona Norte do ...",None,Quatro agentes foram mortos durante uma megaop...,4.0,70
2,3,1318,70,1.00,2025-10-28 00:00:00,"Complexo da Penha, Rio de Janeiro",Rodrigo Nascimento,O policial civil Rodrigo Nascimento morreu nes...,1.0,70
3,4,2119,70,1.00,2025-10-28 00:00:00,"Complexos do Alemão e da Penha, Rio de Janeiro",Marcus Vinícius Cardoso de Carvalho,Uma megaoperação policial nos Complexos do Ale...,70.0,70
4,5,57,4,1.00,2025-12-14 00:00:00,"Campo Grande, Zona Oeste do Rio de Janeiro",Marcus Vinicius Carvalho Cardoso Menesterio,"Um homem, identificado como Marcus Vinicius Ca...",1.0,4
5,6,69,4,1.00,2025-12-14 00:00:00,"Campo Grande, zona oeste do Rio de Janeiro",Marcus Vinicius Carvalho,Um homem foi preso em flagrante por atropelar ...,1.0,4
6,7,57,4,1.00,2025-12-14 00:00:00,"Campo Grande, Rio de Janeiro",Marcus Vinicius Carvalho Cardoso Menesterio,Marcus Vinicius Carvalho Cardoso Menesterio mo...,NaN,4
7,8,276,19,1.00,2025-12-02 00:00:00,"Conjunto do Amarelinho, Acari, Zona Norte do R...",Emerson Costa dos Santos,Um intenso tiroteio entre traficantes do Coman...,2.0,19
8,9,296,19,1.00,2025-12-02 00:00:00,"Acari, Zona Norte, Rio de Janeiro",Emerson Costa dos Santos e Carlos Octavio de A...,"Dois homens, Emerson Costa dos Santos e Carlos...",2.0,19
9,10,364,19,1.00,2025-12-02 00:00:00,"Conjunto do Amarelinho, Acari, Zona Norte do Rio",Emerson Costa do Santos e Carlos Octavio de Ar...,Dois homens foram mortos em um confronto entre...,2.0,19


In [6]:
# First step is to improve the data that we extract from the source

# I want to extract the following information from the source:

# 1. Structured Location Fields
# extracted_neighborhood (String) - Normalized neighborhood name
# extracted_street (String) - Street name/address
# extracted_city (String) - City (default: "Rio de Janeiro")
# extracted_state (String) - State (default: "Rio de Janeiro")
# extracted_country (String) - Country (default: "Brasil")
# Keep extracted_location (String) - Original full location string for reference

# 2. Normalized Victim Information
# extracted_victim_name_normalized (String) - Normalized name (lowercase, no accents, standardized)
# extracted_victim_age (Integer, nullable) - Age if mentioned
# extracted_victim_gender (String, nullable) - Gender if mentioned
# number_of_victims (Integer, nullable) - Number of victims if mentioned

# 3. Additional Identifiers
# extracted_method (String, nullable) - Method of death (e.g., "tiro", "facada", "envenenamento")
# extracted_time_of_day (String, nullable) - Time period if mentioned (e.g., "manhã", "tarde", "noite")

# 4. Date Precision
# extracted_date_precision (String) - Precision level: "exact", "day", "approximate", "unknown"
# Keep extracted_date (DateTime) - Best estimate of event date

In [ ]:
from typing import Dict, List, Optional, Literal

from pydantic import BaseModel, Field, field_validator, model_validator

# ---- Type definitions for standardization ----
# Using Literal instead of Enum for better compatibility with Gemini/Instructor

HomicideType = Literal[
    "Homicídio",
    "Homicídio Qualificado",
    "Homicídio Culposo",
    "Tentativa de Homicídio",
    "Latrocínio",
    "Feminicídio",
    "Infanticídio",
    "Outro"
]

MethodOfDeath = Literal[
    "Arma de fogo",
    "Arma branca",
    "Estrangulamento",
    "Asfixia",
    "Espancamento",
    "Atropelamento",
    "Envenenamento",
    "Objeto contundente",
    "Incêndio",
    "Queda",
    "Outro",
    "Não especificado"
]

# ---- Classes for Structured Extraction ----

class Location(BaseModel):
    """Estrutura de dados de localização extraída da notícia."""
    neighborhood: Optional[str] = Field(
        None, 
        description="Nome do bairro onde ocorreu a morte violenta. Use apenas se explicitamente mencionado."
    )
    street: Optional[str] = Field(
        None, 
        description="Nome da rua, avenida ou logradouro. Exemplo: 'Rua das Flores', 'Avenida Paulista'"
    )
    establishment: Optional[str] = Field(
        None,
        description="Nome do estabelecimento ou tipo de local. Exemplo: 'Residência', 'Via pública', 'Bar e Restaurante', 'Terreno baldio'"
    )
    city: Optional[str] = Field(
        None, 
        description="Cidade onde ocorreu a morte violenta"
    )
    state: Optional[str] = Field(
        None, 
        description="Estado em sigla para Brasil (RJ, SP, MG, etc.) ou nome completo para outros países"
    )
    country: Optional[str] = Field(
        "Brasil", 
        description="País onde ocorreu a morte violenta. Use as informações do texto para inferir o país."
    )
    full_location_description: Optional[str] = Field(
        None, 
        description="Descrição completa e precisa do local."
    )

class IdentifiablePerpetrator(BaseModel):
    """Dados estruturados do autor/suspeito de morte violenta identificável."""
    name: Optional[str] = Field(
        None,
        description="Nome completo do autor, se identificado."
    )
    age: Optional[int] = Field(
        None,
        description="Idade do autor."
    )
    gender: Optional[Literal["masculino", "feminino", "outro", "não informado"]] = Field(
        None,
        description="Gênero do autor."
    )
    occupation: Optional[str] = Field(
        None,
        description="Profissão ou ocupação do autor, se mencionada."
    )
    relationship_to_victim: Optional[str] = Field(
        None,
        description="Relação do autor com a vítima, se mencionada. Exemplo: 'cônjuge', 'desconhecido', 'colega', 'familiar'"
    )
    is_security_force: Optional[bool] = Field(
        None,
        description="Indica se o autor é integrante das forças de segurança pública (Ex: policial militar, policial civil, etc.). True se for, False se não for, None se não mencionado."
    )
    description: Optional[str] = Field(
        None,
        description="Descrição física ou características do autor mencionadas"
    )

class UnidentifiedPerpetratorGroup(BaseModel):
    """Grupo de autores/suspeitos não identificados individualmente."""
    count: int = Field(..., description="Número de autores neste grupo")
    description: str = Field(
        ...,
        description="""Descrição do grupo conforme texto. Ex: "criminosos", "suspeitos", "policiais", "traficantes", "homens armados\""""
    )
    is_security_force: Optional[bool] = Field(
        None,
        description="Este grupo é de forças de segurança?"
    )
    is_civilian: Optional[bool] = Field(
        None,
        description="Este grupo é de civis?"
    )
    context: Optional[str] = Field(
        None,
        description="Contexto adicional sobre este grupo (ex: 'fugiram do local', 'presos durante operação X')"
    )

class Perpetrators(BaseModel):
    """Dados sobre os autores/suspeitos de morte violenta."""
    identifiable_perpetrators: List[IdentifiablePerpetrator] = Field(
        ...,
        description="Lista de autores/suspeitos de morte violenta. Crie uma entrada para cada autor mencionado somente quando houver informações suficientes para identificar o autor."
    )
    number_of_identifiable_perpetrators: int = Field(
        ...,
        description="Número de autores/suspeitos identificados"
    )
    unidentified_groups: Optional[List[UnidentifiedPerpetratorGroup]] = Field(
        None,
        description="Lista de autores/suspeitos não identificados"
    )
    number_of_unidentified_perpetrators: Optional[int] = Field(
        None,
        description="Número de autores/suspeitos não identificados"
    )
    number_of_perpetrators: int = Field(
        ...,
        description="Número total de autores/suspeitos de morte violenta mencionados na notícia"
    )

class IdentifiableVictim(BaseModel):
    """Dados estruturados da vítima de morte violenta."""
    name: Optional[str] = Field(
        None, 
        description="Nome completo da vítima. Se apenas primeiro nome ou apelido, registrar o que foi informado."
    )
    age: Optional[int] = Field(
        None, 
        description="Idade da vítima em anos"
    )
    gender: Optional[Literal["masculino", "feminino", "outro", "não informado"]] = Field(
        None, 
        description="Gênero da vítima inferido do texto"
    )
    occupation: Optional[str] = Field(
        None,
        description="Profissão ou ocupação da vítima, se mencionada"
    )
    relationship_to_perpetrator: Optional[str] = Field(
        None,
        description="Relação com o autor do crime, se mencionada. Exemplo: 'cônjuge', 'desconhecido', 'colega', 'familiar'"
    )
    is_security_force: Optional[bool] = Field(
        None,
        description="Indica se a vítima é integrante das forças de segurança pública (Ex: policial militar, policial civil, guarda municipal, etc.). True se for, False se não for, None se não mencionado."
    )
    description: Optional[str] = Field(
        None,
        description="Descrição física ou características mencionadas"
    )

class UnidentifiedVictimGroup(BaseModel):
    """Grupo de vítimas não identificadas individualmente."""
    count: int = Field(..., description="Número de vítimas neste grupo")
    description: str = Field(
        ...,
        description="""
        Descrição do grupo conforme texto.
        Ex: "moradores", "suspeitos", "policiais", "pessoas", "civis", "criminosos"
        """
    )
    is_security_force: Optional[bool] = Field(
        None,
        description="Este grupo é de forças de segurança?"
    )
    is_civilian: Optional[bool] = Field(
        None,
        description="Este grupo é de civis?"
    )
    context: Optional[str] = Field(
        None,
        description="Contexto adicional sobre este grupo (ex: 'mortos durante operação X')"
    )

class Victims(BaseModel):
    """Dados sobre as vítimas de morte violenta."""
    identifiable_victims: List[IdentifiableVictim] = Field(
        ...,
        description="Lista de vítimas de morte violenta. Crie uma entrada para cada vítima mencionada somente quando houver informações suficientes para identificar a vítima."
    )
    number_of_identifiable_victims: int = Field(
        ...,
        description="Número de vítimas identificadas"
    )
    unidentified_groups: Optional[List[UnidentifiedVictimGroup]] = Field(
        None,
        description="Lista de vítimas não identificadas"
    )
    number_of_unidentified_victims: Optional[int] = Field(
        None,
        description="Número de vítimas não identificadas"
    )
    number_of_victims: int = Field(
        ...,
        description="Número total de vítimas de morte violenta mencionadas na notícia"
    )

class DateVerification(BaseModel):
    """Verificação rigorosa da data antes de extrair."""
    has_explicit_date: bool = Field(
        ...,
        description="""
        O texto contém uma data COMPLETA e EXPLÍCITA (dia/mês/ano OU dia/mês com ano claro no contexto)?
        
        TRUE apenas se houver:
        - "15 de dezembro de 2025"
        - "15/12/2025"
        - "em 12 de março" (se o ano 2024 está claramente estabelecido no contexto)
        
        FALSE se houver apenas:
        - "ontem", "hoje", "na semana passada"
        - "sexta-feira (12)", "segunda-feira (15)" (dia da semana com número mas SEM ano explícito)
        - "há três dias", "recentemente"
        - Qualquer termo relativo
        """
    )
    
    date_text_quote: Optional[str] = Field(
        None,
        description="""
        Se has_explicit_date é TRUE, copie EXATAMENTE o trecho do texto que contém a data completa.
        
        Deve ser uma citação LITERAL do texto original, palavra por palavra.
        Se has_explicit_date é FALSE, deixe como null.
        """
    )
    
    year_explicitly_mentioned: bool = Field(
        ...,
        description="""
        O ANO está explicitamente mencionado no trecho da data?
        
        TRUE: "15 de dezembro de 2025", "12/03/2024"
        FALSE: "sexta-feira (12)", "no dia 15", "em março"
        """
    )
    
    verification_reasoning: str = Field(
        ...,
        description="""
        Explique seu raciocínio sobre a data:
        - O que o texto diz exatamente?
        - Por que você marcou has_explicit_date como TRUE ou FALSE?
        - Se FALSE, por que não é possível extrair a data?
        """
    )

class DateTime(BaseModel):
    """Dados estruturados de data e hora."""
    
    date_verification: DateVerification = Field(
        ...,
        description="Verificação rigorosa se há data explícita no texto"
    )
    
    date: Optional[str] = Field(
        None, 
        description="""
        Data da morte violenta no formato AAAA-MM-DD.
        
        REGRA ABSOLUTA: Este campo DEVE ser null se date_verification.has_explicit_date é FALSE.
        
        Use data SOMENTE se:
        1. date_verification.has_explicit_date é TRUE
        2. date_verification.year_explicitly_mentioned é TRUE
        3. Há uma data completa no formato dia/mês/ano no texto
        
        NUNCA calcule ou infira datas de termos relativos.
        """
    )
    
    date_precision: Optional[Literal["exata", "parcial", "não informada"]] = Field(
        None,
        description="""
        - "exata": data completa (dia/mês/ano) explícita no texto
        - "parcial": apenas dia da semana ou mês mencionado, sem ano
        - "não informada": sem data ou apenas termos relativos
        """
    )
    
    time: Optional[str] = Field(
        None,
        description="""
        Horário específico se explicitamente mencionado no texto.
        
        FORMATOS ACEITOS:
        - Horário exato: "20h30", "15:45", "às 23h"
        - Aproximação explícita: "por volta das 20h", "cerca de 15h"
        
        NÃO USE se apenas período do dia for mencionado ("à noite", "de manhã").
        """
    )
    
    time_of_day: Optional[Literal["madrugada", "manhã", "tarde", "noite", "não informado"]] = Field(
        None, 
        description="""
        Período do dia quando ocorreu a morte violenta, baseado no texto.
        
        Use APENAS se explicitamente mencionado ou se houver horário específico.
        """
    )
    
    @model_validator(mode='after')
    def validate_date_consistency(self):
        """Valida que a data só existe se a verificação permitir."""
        if self.date is not None:
            if not self.date_verification.has_explicit_date:
                raise ValueError(
                    f"ERRO: Campo 'date' está preenchido mas date_verification.has_explicit_date é FALSE. "
                    f"Raciocínio: {self.date_verification.verification_reasoning}"
                )
            if not self.date_verification.year_explicitly_mentioned:
                raise ValueError(
                    f"ERRO: Campo 'date' está preenchido mas date_verification.year_explicitly_mentioned é FALSE. "
                    f"Não é possível extrair data completa sem ano explícito."
                )
        return self

class HomicideDynamic(BaseModel):
    """Dinâmica da morte violenta estruturada."""
    
    title: str = Field(
        ...,
        description="""
        Título técnico da ocorrência seguindo o formato:
        [TIPO DE HOMICÍDIO] - [LOCAL] - [DATA OU "DATA NÃO INFORMADA"]
        
        IMPORTANTE: Se não houver data completa verificada, use "DATA NÃO INFORMADA" no lugar da data.
        
        Exemplos:
        - "HOMICÍDIO QUALIFICADO - VIA PÚBLICA BAIRRO CENTRO - 15/12/2025"
        - "FEMINICÍDIO - RESIDÊNCIA SANTA CRUZ - DATA NÃO INFORMADA"
        - "LATROCÍNIO - ESTABELECIMENTO COMERCIAL - 10/01/2025"
        """
    )
    
    homicide_type: HomicideType = Field(
        ...,
        description="""
        Classificação do tipo de homicídio segundo terminologia jurídica brasileira.
        Valores permitidos:
        - "Homicídio"
        - "Homicídio Qualificado"
        - "Homicídio Culposo"
        - "Tentativa de Homicídio"
        - "Latrocínio"
        - "Feminicídio"
        - "Infanticídio"
        - "Outro"
        """
    )
    
    method: Optional[MethodOfDeath] = Field(
        None,
        description="""
        Método utilizado para causar a morte violenta.
        Valores permitidos:
        - "Arma de fogo"
        - "Arma branca"
        - "Estrangulamento"
        - "Asfixia"
        - "Espancamento"
        - "Atropelamento"
        - "Envenenamento"
        - "Objeto contundente"
        - "Incêndio"
        - "Queda"
        - "Outro"
        - "Não especificado"
        """
    )
    
    chronological_description: str = Field(
        ...,
        description="""
        Descrição cronológica OBJETIVA dos fatos em linguagem técnica policial.
        
        DEVE:
        - Usar terceira pessoa e voz passiva
        - Linguagem formal, técnica e impessoal
        - Ordem cronológica clara dos eventos
        - Apenas fatos verificáveis no texto
        - Terminologia jurídica adequada
        - Identificar claramente: vítima(s), autor(es), testemunha(s)
        - Se data completa não disponível, use "em data não especificada" ou "em [dia da semana/período mencionado]"
        
        NÃO DEVE:
        - Incluir opiniões ou juízos de valor
        - Usar adjetivos sensacionalistas ("brutal", "covarde", etc.)
        - Especular sobre motivações não declaradas
        - Usar linguagem coloquial ou emotiva
        - Incluir informações não verificadas no texto
        - Inventar datas completas
        """
    )
    

class ViolentDeathEvent(BaseModel):
    """Informações estruturadas completas sobre morte violenta extraída de notícia."""
    
    location_info: Location = Field(
        ..., 
        description="Informações estruturadas do local onde ocorreu a morte violenta"
    )
    
    date_time: DateTime = Field(
        ..., 
        description="Informações de data e hora da morte violenta COM VERIFICAÇÃO RIGOROSA"
    )
    
    victims: Victims = Field(
        ..., 
        description="Dados sobre as vítimas de morte violenta."
    )
    
    perpetrators: Optional[Perpetrators] = Field(
        None,
        description="Lista de autores/suspeitos da morte violenta, se identificados"
    )
    
    homicide_dynamic: HomicideDynamic = Field(
        ...,
        description="Dinâmica completa da morte violenta incluindo título, classificação e descrição técnica"
    )
    
    additional_context: Optional[str] = Field(
        None,
        description="Contexto adicional relevante que não se enquadra nas categorias acima"
    )
    


def extract_violent_death_from_news(
    content: str, 
    model_id: str = "gemini-2.5-flash",
    instructor_client=None
) -> ViolentDeathEvent:
    """
    Extrai informações estruturadas sobre morte violenta de artigo de notícia usando Instructor.
    Implementa Chain of Verification para garantir que datas não sejam inventadas.
    
    Args:
        content: Texto da notícia sobre morte violenta
        model_id: ID do modelo a usar
        instructor_client: Cliente Instructor opcional
    
    Returns:
        ViolentDeathEvent: Objeto Pydantic com todas as informações estruturadas sobre a morte violenta
    """
    # Use provided client or global client
    if instructor_client is None:
        instructor_client = client

    if instructor_client is None:
        raise ValueError("Instructor client not initialized. Please set GEMINI_API_KEY in .env file.")

    # Create a new client with the specified model if different
    instructor_client = instructor.from_provider(
        f"google/{model_id}",
        api_key=GEMINI_API_KEY
    )

    # System prompt to guide the extraction
    system_prompt = """
    Você é um assistente especializado em extrair informações de notícias sobre mortes violentas 
    e convertê-las em descrições técnicas seguindo padrões profissionais de escrivães 
    de polícia no Brasil.

    PRINCÍPIOS FUNDAMENTAIS:
    1. Use APENAS informações explicitamente presentes no texto
    2. NUNCA invente, calcule ou infira informações
    3. Para campos opcionais, deixe null se a informação não estiver disponível
    4. Mantenha objetividade e neutralidade absoluta
    5. Use terminologia jurídica formal e precisa

    REGRA CRÍTICA SOBRE DATAS - LEIA COM ATENÇÃO:
    
    Você DEVE preencher o campo date_verification PRIMEIRO, antes de qualquer extração de data.
    
    O campo date_verification funciona como um VERIFICADOR que impede datas inventadas:
    
    1. has_explicit_date = TRUE SOMENTE se o texto contém data COMPLETA (dia/mês/ano)
       Exemplos de TRUE:
       - "15 de dezembro de 2025"
       - "em 12 de março de 2024"
       - "no dia 20/11/2025"
       
    2. has_explicit_date = FALSE se o texto tem apenas:
       - Dias da semana: "sexta-feira (12)", "na segunda-feira (15)"
       - Termos relativos: "ontem", "hoje", "na semana passada"
       - Períodos: "recentemente", "há três dias"
       
    3. Se has_explicit_date = FALSE, o campo date DEVE ser null
       Não há exceções. O validador vai rejeitar se você tentar preencher date quando has_explicit_date é FALSE.
       
    4. year_explicitly_mentioned deve ser TRUE apenas se o ANO aparece no texto da data
    
    5. verification_reasoning deve explicar claramente por que você decidiu TRUE ou FALSE
    
    IMPORTANTE: É MELHOR deixar date como null do que inventar uma data.
    Dados incompletos são preferíveis a dados falsos.
    
    SOBRE TÍTULOS:
    - Se não há data completa verificada, use "DATA NÃO INFORMADA" no título
    - Exemplo: "FEMINICÍDIO - RESIDÊNCIA SANTA CRUZ - DATA NÃO INFORMADA"
    """

    event = instructor_client.create(
        response_model=ViolentDeathEvent,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": content}
        ],
        max_retries=3
    )

    return event

In [125]:
# Test extraction with Instructor
result = extract_violent_death_from_news(
    dfs['source'].loc[dfs['source']['id'] == 133, 'content'].iloc[0],
    model_id="gemini-2.5-flash"
)

print(result.model_dump_json(indent=2))

{
  "location_info": {
    "neighborhood": "Nova Aurora",
    "street": null,
    "establishment": "Morro do Avião",
    "city": "Belford Roxo",
    "state": "RJ",
    "country": "Brasil",
    "full_location_description": "bairro Nova Aurora, em Belford Roxo, na Baixada Fluminense, no Rio de Janeiro (RJ), próximo ao Morro do Avião"
  },
  "date_time": {
    "date_verification": {
      "has_explicit_date": false,
      "date_text_quote": null,
      "year_explicitly_mentioned": false,
      "verification_reasoning": "O texto menciona 'Na tarde deste sábado (02)', que é um dia da semana com um número, mas não uma data completa (dia/mês/ano). Portanto, não é uma data explícita."
    },
    "date": null,
    "date_precision": "parcial",
    "time": null,
    "time_of_day": "tarde"
  },
  "victims": {
    "identifiable_victims": [
      {
        "name": null,
        "age": null,
        "gender": "não informado",
        "occupation": null,
        "relationship_to_perpetrator": "desconh

In [114]:
print(dfs['source'].loc[dfs['source']['id'] == 1509, 'content'].iloc[0])


KALIL DE OLIVEIRA
FLORIANÓPOLIS, SC (FOLHAPRESS)
O Complexo da Maré, no Rio de Janeiro, amanheceu nesta quarta-feira (17) com uma nova operação policial. A Polícia Civil busca integrantes do Comando Vermelho que estariam envolvidos em crimes como comércio ilegal de armas de fogo e munições, tráfico de drogas e associação criminosa.
Moradores relatam troca de tiros na região do Parque União e da Nova Holanda. As buscas, que são parte de nova fase da Operação Contenção, também acontecem em outros pontos da zona norte e da região dos Lagos. A ação tem participação da Core (Coordenadoria de Recursos Especiais) e DGPE (Delegacia-Geral de Polícia Especializada).
Perfis em redes sociais mostram, desde cedo, viaturas na avenida Brasil, além de relatar a presença de muitos agentes da polícia na região.
A Polícia Civil diz ter identificado núcleos criminosos especializados na circulação de armamentos de alto poder ofensivo, na logística de munições e no abastecimento do tráfico. “Parte significa

133
1509 --> 122 mortos

In [ ]:
# Simple Violent Death Classifier
# Uses a lighter model for fast classification

from pydantic import BaseModel, Field
from typing import Literal

# Use a faster/lighter model for simple classification
# Note: gemini-2.0-flash-lite is the lightweight model for quick classification tasks
SELECTION_MODEL = "gemini-2.0-flash-lite"

class ViolentDeathClassification(BaseModel):
    """Classification result for whether news is about a violent death."""
    
    is_violent_death: bool = Field(
        ...,
        description="""
        TRUE if the news article is about one or more violent deaths (homicides, murders, killings).
        
        Examples of TRUE:
        - Someone was shot and killed
        - A body was found with signs of violence
        - Multiple people died in a shootout
        - Police operation resulted in deaths
        - Someone was stabbed to death
        
        Examples of FALSE:
        - Natural death (illness, old age)
        - Accidents without criminal intent (car crash without murder)
        - Missing persons without confirmed death
        - General violence without deaths (robbery, assault without fatalities)
        - News about violence prevention, security policies
        """
    )
    
    confidence: Literal["alta", "média", "baixa"] = Field(
        ...,
        description="""
        Confidence level in the classification:
        - "alta": Clear case, explicit mention of death by violence
        - "média": Death mentioned but circumstances unclear
        - "baixa": Ambiguous, might be referring to past events or general context
        """
    )
    
    reasoning: str = Field(
        ...,
        description="Brief explanation (1-2 sentences) of why this classification was made."
    )


def classify_violent_death(
    content: str,
    model_id: str = SELECTION_MODEL,
    api_key: str = None,
) -> ViolentDeathClassification:
    """
    Quickly classifies if a news article is about violent death.
    
    Args:
        content: Text of the news article
        model_id: Model to use for classification (default: lighter model)
        api_key: Gemini API key (optional, uses GEMINI_API_KEY env var if not provided)
    
    Returns:
        ViolentDeathClassification with is_violent_death, confidence, and reasoning
    """
    # Use provided key or global
    key = api_key or GEMINI_API_KEY
    if not key:
        raise ValueError("GEMINI_API_KEY not provided")
    
    # Create client for the selection model
    classifier_client = instructor.from_provider(
        f"google/{model_id}",
        api_key=key
    )
    
    system_prompt = """
    Você é um classificador de notícias. Sua única tarefa é determinar se uma notícia 
    trata de uma ou mais MORTES VIOLENTAS (homicídios, assassinatos, execuções).
    
    CLASSIFIQUE COMO MORTE VIOLENTA (is_violent_death = true):
    - Pessoa morta por arma de fogo
    - Pessoa morta por arma branca
    - Corpo encontrado com sinais de violência
    - Morte em operação policial
    - Morte em confronto entre facções/criminosos
    - Feminicídio, latrocínio, homicídio
    
    NÃO CLASSIFIQUE COMO MORTE VIOLENTA (is_violent_death = false):
    - Morte natural (doença, idade)
    - Acidente de trânsito sem intenção criminosa
    - Pessoa desaparecida (sem confirmação de morte)
    - Violência sem morte (assalto, agressão sem óbito)
    - Notícias sobre políticas de segurança ou prevenção
    - Referências a mortes passadas apenas como contexto histórico
    
    Seja objetivo e baseie-se apenas no texto fornecido.
    """
    
    result = classifier_client.create(
        response_model=ViolentDeathClassification,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Classifique esta notícia:\n\n{content}"}
        ],
        max_retries=2
    )
    
    return result

print("Classifier ready! Using model:", SELECTION_MODEL)


In [ ]:
# Test the classifier with a few examples

# Pick some source IDs to test
test_ids = [133, 1509, 57]  # Known violent death articles

print("=" * 80)
print("TESTING VIOLENT DEATH CLASSIFIER")
print("=" * 80)

for source_id in test_ids:
    content = dfs['source'].loc[dfs['source']['id'] == source_id, 'content'].iloc[0]
    
    print(f"\n--- Source ID: {source_id} ---")
    print(f"Content preview: {content[:200]}...")
    print()
    
    result = classify_violent_death(content)
    
    print(f"🔍 Is Violent Death: {result.is_violent_death}")
    print(f"📊 Confidence: {result.confidence}")
    print(f"💭 Reasoning: {result.reasoning}")
    print("-" * 40)


In [ ]:
# Batch test on a sample of sources
# Let's classify a random sample and see the distribution

import random

# Get a sample of sources
sample_size = 10
all_source_ids = dfs['source']['id'].tolist()
sample_ids = random.sample(all_source_ids, min(sample_size, len(all_source_ids)))

results = []

print(f"Classifying {len(sample_ids)} random sources...\n")

for source_id in sample_ids:
    try:
        content = dfs['source'].loc[dfs['source']['id'] == source_id, 'content'].iloc[0]
        if not content or len(content) < 50:
            continue
            
        result = classify_violent_death(content)
        results.append({
            'source_id': source_id,
            'is_violent_death': result.is_violent_death,
            'confidence': result.confidence,
            'reasoning': result.reasoning,
            'content_preview': content[:100] + "..."
        })
        
        status = "✅ VIOLENT DEATH" if result.is_violent_death else "❌ NOT VIOLENT DEATH"
        print(f"[{source_id}] {status} ({result.confidence})")
        
    except Exception as e:
        print(f"[{source_id}] Error: {e}")

# Summary
violent_count = sum(1 for r in results if r['is_violent_death'])
print(f"\n{'='*40}")
print(f"SUMMARY: {violent_count}/{len(results)} classified as violent death")
print(f"{'='*40}")


In [ ]:
# View results as DataFrame
results_df = pd.DataFrame(results)
results_df
